# Fill NaN values

The undefined values in the grids do not allow interpolation of values located in the neighborhood. This behavior is a concern when you need to interpolate values near the land/sea mask of some maps. For example:

<img src="masked_values.svg" alt="Drawing" align="middle" style="width: 300px;"/>

For example, in the figure above, if you want to interpolate the black point with a bilinear interpolation, the undefined red value, set to NaN, will not allow its calculation (the result of the arithmetic operation using a value equal to NaN is NaN). To overcome this problem, the library provides methods to fill undefined values.

---
**Note**

In the case of an interpolation of the nearest neighbor the undefined values have no impact because no arithmetic operation is done on the grid values: we just return the value of the nearest point.

---

## Loess

The [first](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.fill.html#pyinterp.fill.loess) method applies a weighted local regression to extrapolate the boundary between defined and undefined values. The user must indicate the number of pixels on the X and Y axes to be considered in the calculation.

Let's start by building the object handling our grid.

In [ ]:
import xarray as xr
import pyinterp.backends.xarray
import pyinterp.fill

ds = xr.load_dataset("../tests/dataset/mss.nc")
grid = pyinterp.backends.xarray.Grid2D(ds.data_vars["mss"])
filled = pyinterp.fill.loess(grid, nx=3, ny=3)

The function filling the holes near the mask is called

In [ ]:
filled = pyinterp.fill.loess(grid, nx=3, ny=3, num_threads=4)

The image below illustrates the result:

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
%matplotlib inline

fig = plt.figure(figsize=(18, 9))
ax = fig.add_subplot(121, projection=ccrs.PlateCarree(central_longitude=180))
lons, lats = np.meshgrid(grid.x, grid.y, indexing='ij')
ax.pcolormesh(lons, lats, grid.array, transform=ccrs.PlateCarree(), cmap='jet')
ax.coastlines()
ax.set_title("Original MSS")
ax.set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())

ax = fig.add_subplot(122, projection=ccrs.PlateCarree(central_longitude=180))
ax.pcolormesh(lons, lats, filled, transform=ccrs.PlateCarree(), cmap='jet')
ax.coastlines()
ax.set_title("MSS modified using the LOESS filter")
ax.set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())

## Gauss-Seidel

The [second method](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.fill.html#pyinterp.fill.gauss_seidel) consists of replacing all undefined values (NaN) in a grid using the Gauss-Seidel method by relaxation. This [link](https://math.berkeley.edu/~wilken/228A.F07/chr_lecture.pdf) contains more information on the method used.

In [ ]:
has_converged, filled = pyinterp.fill.gauss_seidel(grid)

The image below illustrates the result:

In [ ]:
fig = plt.figure(figsize=(18, 9))
ax = fig.add_subplot(121, projection=ccrs.PlateCarree(central_longitude=180))
lons, lats = np.meshgrid(grid.x, grid.y, indexing='ij')
ax.pcolormesh(lons, lats, grid.array, transform=ccrs.PlateCarree(), cmap='jet')
ax.coastlines()
ax.set_title("Original MSS")
ax.set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())

ax = fig.add_subplot(122, projection=ccrs.PlateCarree(central_longitude=180))
ax.pcolormesh(lons, lats, filled, transform=ccrs.PlateCarree(), cmap='jet')
ax.coastlines()
ax.set_title("MSS modified using Gauss-Seidel")
ax.set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())